# Imports

In [2]:
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import numpy as np
import seaborn as sns
import glob
import pickle
import corner
import scipy.optimize
import re
import os
import sys
import utils
import importlib # for refreshing changes made to utils

#for loading TEPSPEC_Util scripts into trace pkl
sys.path.append('/Users/mango/tepspec/IMACS/')

from astropy.io import fits, ascii
from astropy.table import Table
from astropy.time import Time
from collections import Counter, namedtuple
from matplotlib.colors import LogNorm
from matplotlib.widgets import Slider
from scipy import stats
from collections import OrderedDict

sns.set_context('paper')
sns.set_style('darkgrid')
sns.set_color_codes('deep')
sns.set_palette('deep')
%matplotlib qt5
plt.ion()

# Wavelength Calibration

Some of the lamp spectra through the target and comparison star slits are completely shifted off of the reference lamp spectra, so guess_lines.py can't be used. Instead, the lines will be indentified manually from http://www.lco.cl/telescopes-information/magellan/instruments/imacs/imacs-spectral-atlas/imacs-spectral-atlas.

To make things a little easier, this routine will will automatically record the pixel coordinate from a mouse press to save me an extra step in writing down the pixel/wavelength values.

In [ ]:
# read in desired lamp spec
name = 'WASP43b_8'
path = '../wavelength_calibration/arcs_ut150224/stage/' + name + '_arc.fits'
#path = '/Users/mango/Desktop/HeNeAr_gris300.fits'
data = fits_data(path)

pix_loc = []

# display and record pix location on 'x' key press
def on_key(event): 
    if (event.key == 'x' or event.key == 'X') and event.inaxes == ax:
        x = event.xdata
        y = event.ydata
        
        plt.axvline(x, c='r', ls='--', lw=0.5)
        ax.plot(x, y, 'rX', ms=10)
        x = str( np.round(x, 4) )
        pix_loc.append(x)
        
    fig.canvas.draw()

fig, ax = plt.subplots(figsize = (15,4))

#ax.cla()
ax.plot(data)
ax.set_title(name)
ax.set_xlabel('pixel')
ax.set_ylabel('flux (arbitrary units)')
#[ax.axvline(xc,lw=0.5,ls ='--',c='r',alpha=0.5) for xc in arc_lines['pix']]
    
fig.tight_layout()
cid = fig.canvas.mpl_connect('key_press_event', on_key)

In [ ]:
f = open('/Users/mango/Desktop/test.txt', 'wb')
for pix in pix_loc:
    f.write("%s\n" % pix)
f.close()

In [ ]:
for f in glob.glob('../wavelength_calibration/arcs_ut170410/comp*line*'):
    t = ascii.read(f)
    l = f.split('/')[-1]
    plt.plot(t['Pix'], t['Wav'], 'o', label=l, alpha=0.5)
    
plt.legend(loc='best')

### noao line lines

In [ ]:
plt.figure(figsize=(15,4))
arc_lines = ascii.read('../wavelength_calibration/noao/ref_linelist.txt')
hdu = fits.open('../wavelength_calibration/noao/noao.fits')

data = hdu[0].data
hdu.close()

wavs = arc_lines['Fit']
xs = arc_lines['Pixel']
ys = []
for x_guess in xs:
    ys.append(data[int(x_guess)])
plt.plot(data, c='orange')
[plt.axvline(xc, lw=0.5, ls = '--', c='b', alpha=0.5) for xc in xs]
[plt.annotate(str(wav), (s_x, s_y)) for wav, s_x, s_y in zip(wavs, xs, ys)]

plt.title('noao reference')
plt.xlabel('pixel')
plt.ylabel('arbitrary flux')

plt.tight_layout()

plt.show()

### lco lamp lines

In [ ]:
plt.figure(figsize=(15,4))
fpath = '../wavelength_calibration/lco_lines.txt'
arc_lines = ascii.read(fpath)

fpath = '../wavelength_calibration/HeNeAr_gris300.fits'
data = fits_data(fpath)

wavs = arc_lines['wav']
xs = arc_lines['pix']
ys = []
for x_guess in xs:
    ys.append(data[int(x_guess)])
plt.plot(data, c='orange')
[plt.axvline(xc, lw=0.5, ls = '--', c='b', alpha=0.5) for xc in xs]
[plt.annotate(str(wav), (s_x, s_y)) for wav, s_x, s_y in zip(wavs, xs, ys)]

plt.title('lco reference')
plt.xlabel('pixel')
plt.ylabel('arbitrary flux')

plt.tight_layout()

plt.show()

### calibration file check

In [ ]:
def cal_plot(ax, guess_path, data_path, c):
    guess_lines = ascii.read(guess_path)

    data = fits_data(data_path)

    wavs = guess_lines['Wav']
    xs = guess_lines['Pix']
    ys = []
    for x_guess in xs:
        ys.append(data[int(x_guess)])

    ax.plot(data, c=c)
    [ax.axvline(xc, lw=0.5, ls = '--', c='k', alpha=0.5) for xc in xs]
    [ax.annotate(str(wav), (s_x, s_y)) for wav, s_x, s_y in zip(wavs, xs, ys)]

    ax.set_xlabel('pixel')
    ax.set_ylabel('arbitrary flux')
    
    title = '{}: {} + {}'.format(guess_path.split('/')[-2],
        guess_path.split('/')[-1], data_path.split('/')[-1])
    ax.set_title(title)

In [ ]:
fig, ax = plt.subplots(figsize=(15,4))

guess_path = '../wavelength_calibration/arcs_ut150309/comp1_lines_chips.txt'
data_path = '../wavelength_calibration/arcs_ut150309/comp1_2_arc.fits'

cal_plot(ax, guess_path, data_path, 'orange')

fig.tight_layout()

### Compare Calibrations between datasets

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(15,6), sharex=True, sharey=True)

guess_path = '../wavelength_calibration/arcs_ut150224/comp6_lines_chips.txt'
data_path = '../wavelength_calibration/arcs_ut150224/comp6_2_arc.fits'
cal_plot(ax[0], guess_path, data_path, 'b')

guess_path = '../wavelength_calibration/arcs_ut150309/comp6_lines_chips.txt'
data_path = '../wavelength_calibration/arcs_ut150309/comp6_2_arc.fits'
cal_plot(ax[1], guess_path, data_path, 'r')

fig.tight_layout()

### Overview Line List Inspection

In [ ]:
fig, axis = plt.subplots(2, 4, figsize = (14,5))

ax_list = []
for i in range(2):
    for j in range(4):
        ax_list.append(axis[i][j])

#read in spectra and line lists
spec_list = []
line_lists = []
title_list = []
for i in glob.glob('../wavelength_calibration/arcs_ut150224/stage/*arc.fits'):
    hdu = fits.open(i)
    spec = hdu[0].data # data
    spec_list.append(spec)
    title_list.append(i.split('/')[-1])
    hdu.close()
    
for i in glob.glob('../wavelength_calibration/arcs_ut150224/*lines*txt'):
    lines = ascii.read(i)
    line_lists.append(lines)

k = 0
z = 0
xs_list = []
for ax, spec, lines, ti in zip(ax_list, spec_list, line_lists, title_list):
    wavs = lines['Wav']
    xs = lines['Pix']
    xs_list.append(xs)
    ys = []
    for x_guess in xs:
        ys.append(spec[int(x_guess)])

    ax.plot(spec, c='orange')
    ax.set_title(ti)
    [ax.axvline(xc, lw=0.5, ls = '--', c='b', alpha=0.5) for xc in xs]
    for wav, s_x, s_y in zip(wavs, xs, ys):
        ax.annotate(str(wav), (s_x, s_y), fontsize=12) 

    i = 0

def on_key(event): # pan and zoom each plot key press
    global i
    if (event.key == 'n' or event.key == 'N'):
        for ax, spec, lines in zip(ax_list, spec_list, line_lists):
            wavs = lines['Wav']
            xs = lines['Pix']
            xs_list.append(xs)
            ys = []
            for x_guess in xs:
                ys.append(spec[int(x_guess)])

            ax.set_xlim(xs[i] - 50, xs[i] + 50)
            ax.set_ylim(ys[i]*-0.1, ys[i]*3)

        i += 1
        if (i > len(xs) - 1): # go back to beginning after reaching end
            i = 0
            

    fig.canvas.draw()


fig.tight_layout()
cid = fig.canvas.mpl_connect('key_press_event', on_key)

### Line List Check

In [ ]:
def plot_line_pix(ax, lines_chips_path):
    data = ascii.read(lines_chips_path)
    wav, pix = data['Wav'], data['Pix']
    
    ax.plot(wav, pix, label=lines_chips_path.split('/'))
    
    return ax

In [ ]:
fig, ax = plt.subplots()

#fnames = glob.glob('../wavelength_calibration/arcs_ut150224/*lines*.txt')
fnames = glob.glob('../wavelength_calibration/arcs_ut150*/comp*_lines*')

p = [plot_line_pix(ax, fname) for fname in fnames]

ax.legend()

# File Inspection

In [ ]:
plt.figure()
fpath = '../WASP43/ut170410/tepspec_final/custom/mfilt/x_bad_pixels_c8.fits'
x_bad = utils.fits_data(fpath)
fpath = fpath.replace('x_bad', 'y_bad')
y_bad = utils.fits_data(fpath)

plt.plot(x_bad, y_bad, 'bo', alpha=0.5)

fpath = '../WASP43/ut170410/tepspec_final/custom/mfilt_DHU/x_bad_pixels_c8.fits'
x_bad = utils.fits_data(fpath)
fpath = fpath.replace('x_bad', 'y_bad')
y_bad = utils.fits_data(fpath)

plt.plot(x_bad, y_bad, 'r.', alpha=0.5)

### trace check

In [ ]:
# XX, YY.pkl has shape (time x # trace points)
# select dataset
date = 'ut150224'
binsize = '75nm'

# load in data
fpath = '../WASP43/{}/tepspec_final/{}/XX.pkl'.format(date, binsize)
tr_x = utils.pkl_load(fpath)
fpath = fpath.replace('XX', 'YY')
tr_y = utils.pkl_load(fpath)

# plot
fig, axes = plt.subplots(2, 5, figsize=(8,5), sharey=False)

cmap_comps ='Blues'
cmap_targ = 'Oranges'
n_sample = 33 # number of time indices to sample
colors_comps = np.array(sns.color_palette(cmap_comps, n_sample))
colors_targ = np.array(sns.color_palette(cmap_targ, n_sample))


# sort keys
objs = []
for obj in list(tr_x.keys()):
    objs.append(obj.decode('UTF-8')) # Python 3 byte string stuff
objs.sort()

# sample n time indices (rounded down to nearest integer) 
# from the N total sci images 
for ax, obj in zip(axes.flatten(), objs):
    obj = obj.encode('UTF-8') # back to bytes
    # t_idx should be the same for all objs, but just in case
    t_idx = np.linspace(0, len(tr_x[obj])-1, n_sample, dtype=int)
    if (b'WASP' in obj): colors = colors_targ
    else: colors = colors_comps
    for t, c in zip(t_idx, colors):
        ax.plot(tr_x[obj][t], tr_y[obj][t], '.', c=c, label=t, alpha=0.5) 
        ax.set_title(obj.decode('utf-8'))

fig.tight_layout()
#plt.legend()
plt.show()

### Objects_trace.pkl

In [ ]:
#import TEPSPEC_Util
fpath = '../WASP43/ut150224/tepspec_final/Objects_trace.pkl'
obj_trace = pkl_load(fpath)

In [ ]:
fpath = '../WASP43/ut170410/tepspec_final/75nm/Objects_trace.pkl'
obj_trace_ut170410 = pkl_load(fpath)

In [ ]:
obj_trace_ut170410[b'WASP43b_8'].chip

### spec files

In [ ]:
fpath = '../WASP43/ut150224/tepspec_final/75nm/mfilt/WASP43b_8_spec.fits'
#fpath = '../WASP43/ut150309/tepspec_final/75nm/WASP43b_8_spec.fits'
#fpath = '../WASP43/ut170410/tepspec_final/custom/WASP43b_8_spec.fits'

w43 = utils.fits_data(fpath)

plt.figure()
sns.set_palette('deep')
for t, spec in enumerate(w43[:200,6,:]):
    #if np.sum(spec != 0): print(t)
    plt.plot(spec, alpha=0.25, c='b')

#plt.title('WASP43 Spectrum 75nm with ut150309 bad pixel mask, opt extract')
plt.xlabel('pixel')
plt.ylabel('count')
#plt.savefig('/Users/mango/Desktop/after_optimal.png', bbox_inches='tight')

In [ ]:
"""
plots items in <object>_spec.fits files.
has shape time x spec_item x ypix [it's ~ 2048] (|| to wav direction)
spec_item:
0: Wavelength
1: Simple extracted object spectrum
2: Simple extracted flat spectrum
3: Pixel sensitivity (obtained by the flat)
4: Simple extracted object spectrum/pixel sensitivity
5: Sky flag (0 = note uneven sky, 1 = probably uneven profile, 
   2 = Sky_Base failed)
6: Optimally extracted object spectrum
7: Optimally extracted object spectrum/pixel sensitivity
"""
def spec_plot(spec_item):
    fig, axs = plt.subplots(1, 2, figsize=(10,3))
    
    objs = ['comp4_4', 'comp4_4']
    for ax, obj in zip(axs, objs):
        hdu = fits.open('/Users/mango/Desktop/'+obj+'_spec.fits')
        data = hdu[0].data
        hdu.close()
        
        # data[number of sci frames, spec_items, 
        # pixel height of spectrum (2048)]
        # by 'height' I mean in spectral direction
        for i in range(len(data[:,0,0])):
            ax.plot(data[i,spec_item,:])
        
        #ax.set_title(obj.split('_')[-1])
        
    fig.tight_layout()
    fig.show()

spec_plot(0)

### File Inspection

In [ ]:
path_list = glob.glob(
    '/Users/mango/data/ACCESS/IMACS/WASP43/ut150224/*0*.fits')

filenames = []
objects = []
slts = []
for path in path_list:
    with open(path, 'rb') as f:
        hdu = fits.open(f)
        filename = hdu[0].header['FILENAME']
        obj     = hdu[0].header['OBJECT']
        slt     = hdu[0].header['SLITMASK']
    #airmass = hdu[0].header['AIRMASS']
    #am.append(airmass)
        if ('c8' in filename): # other chips redundant for ID purposes
    #    object_dict.update({filename:obj})
            filenames.append(filename)    
            objects.append(obj)
            slts.append(slt)
#for key in sorted(object_dict.iterkeys()):
#    print key, object_dict[key]

In [ ]:
t = Table()
t['fnames'] = filenames
t['type'] = objects
t['slit mask'] = slts

t.sort('fnames')
t

In [ ]:
plt.plot(object_dict.values(), '.')

In [ ]:
import re
sc_counter = 0
sci_frame = []
for k, v in object_dict.iteritems():
    if 'science' in v:
        sc_counter += 1z
        file_num = re.split('t|c', k) # get number portion of filename
        sci_frame.append(file_num[1])
sci_frame = np.sort(sci_frame)
print sc_counter

### For adding full flats from WASP-19 to WASP-43 ut170410 dataset

In [ ]:
full_flats = []
for i, j in object_dict.items():
    if 'full' in j and 'c8' in i:
        print i,j
        full_flats.append(i)

#### Airmass filter

In [ ]:
path_list = glob.glob(
    '/data/ACCESS/IMACS/WASP43/ut150224/*0*.fits')

filename_list = []
for path in path_list:
    hdu = fits.open(path)
    filename = hdu[0].header['FILENAME']
    airmass = hdu[0].header['AIRMASS']
    obj     = hdu[0].header['OBJECT']
    hdu.close()
    #if obj == 'science image':
    #    i += 1
    #print obj, filename
    if ((airmass >= 1.5) and ('c8' in filename) and 'sci' in obj): #and (obj == 'science image'):
        #filename_list.append(filename)
        print filename, obj
    #if ('science frame wasp43' in obj and 'c8' in filename):
    #    filename_list.append(filename)

In [ ]:
fig, ax = plt.subplots(1,1)

ax.plot([1,2,3,4])
ax.axvline(1)
fig.show()

### Aperture Check

#### Interactive 1D profile plotter to verify apertures used in tepspec

In [ ]:
path = '/Users/mango/data/ACCESS/IMACS/WASP43/ut150224/ift0250c8.fits'
hdu = fits.open(path)
name = hdu[0].header['filename']
data = hdu[0].data
ut_start = hdu[0].header['UT-TIME']
ut_end = hdu[0].header['UT-END']
hdu.close()

fig, axes = plt.subplots(1,2, figsize=(8, 6))

im = axes[0].imshow(data, cmap='CMRmap', vmin=600, vmax=30000)
plt.colorbar(im, ax=axes[0])
axes[0].grid(False) # get rid of gridlines on imshow
#axes.set_title(name + ' -- sci frame 25')
axes[0].set_xlabel('%s - %s' % (ut_start, ut_end))
axes[1].set_ylim(0, 25000)

#plt.savefig('/Users/mango/Desktop/ut150309_image.png', bbox_inches='tight')

def drag(event): # mouse drag
    x = event.xdata
    y = event.ydata
    
    if (event.inaxes == axes[0]): # only detect mouse drags in imshow plot
        axes[1].cla()
        axes[1].plot(data[int(round(y)), :])
        axes[1].set_xlim(x - 50, x + 50)
        axes[1].set_ylim(0, 30000)
        axes[1].set_ylabel('counts')
    
    fig.tight_layout()
    fig.canvas.draw()
    
cid = fig.canvas.mpl_connect('motion_notify_event', drag)

# tepspec

## Raw WLC Summary

In [9]:
importlib.reload(utils)

# load tepspec LC pkl
target = 'WASP43'
date = 'ut170410'
binsize = 'bintest'

fpath = '../{0}/{1}/tepspec/{2}/LC*.pkl'
fpath = fpath.format(target, date, binsize)
print(fpath)

fpath = glob.glob(fpath)[0]

LC = utils.pkl_load(fpath)
fig, ax = plt.subplots()

p = utils.plot_raw_LCs(ax, LC, plot_date=True)

p.legend()
p.set_title(f'{date} - {binsize}')

fig.tight_layout()

../WASP43/ut170410/tepspec/bintest/LC*.pkl


## WLC / Comp Stars

In [14]:
fig, ax = plt.subplots(1, 1, figsize=(8,5))
importlib.reload(utils)

# load tepspec LC pkl 
target = 'WASP43'
date = 'ut170410'
binsize = 'bintest'
fpath = '../{0}/{1}/tepspec/{2}/LC*.pkl'
fpath = fpath.format(target, date, binsize)
fpath = glob.glob(fpath)[0]
#LC = utils.pkl_load(fpath)
# get derived values from above pkl for plotting
comps_to_use = ['comp1','comp2', 'comp3', 'comp4', 'comp6']
#comps_to_use = ['comp1','comp2']
LC_comp_div = utils.get_comp_divLC(LC, comps_to_use) 

# plot
p = utils.plot_LC_comp_div(ax, LC_comp_div.t, LC_comp_div.comp_div, 
                           comps_to_use, target, date, binsize, utc=False,
                          ls = 'none', marker='.')
p.set_title('{} {} {} Divided WLC'.format(target, date, binsize))
#p.set_ylim(0.97, 1.01)

Text(0.5,1,'WASP43 ut170410 bintest Divided WLC')

### Binned Flux

In [16]:
importlib.reload(utils)
target = 'WASP43'
date= 'ut170410'
offs = 0.025
cmap = 'Spectral_r'
binsize = 'binsize'

fpath = '../{0}/{1}/tepspec/{2}/LC*.pkl'
fpath = fpath.format(target, date, binsize)
fpath = glob.glob(fpath)[0]
#LC = utils.pkl_load(fpath)
comps_to_use = ['comp1', 'comp2', 'comp3', 'comp4', 'comp6']
#comps_to_use = ['comp1', 'comp2']
LC_w = utils.get_comp_divLC_w(LC, comps_to_use)

# get t0 from WLC
method = 'poly'
binsize = '75nm'
fpath = f'../WASP43/{date}/{method}_WLC/{binsize}/final_LCs.pkl'
wlc = utils.pkl_load(fpath)

# filter out bad points
delete_idx = []
t_idx = range(len(LC_w.t))
fit_idx = np.delete(t_idx, delete_idx)
time = (LC_w.t[fit_idx] - wlc['t0'])*24
flux = LC_w.flux[:, fit_idx]
resid = LC_w.resid_wlc[:, fit_idx]

fig, ax = plt.subplots(1, 2, figsize=(6,8), sharex=True, sharey=True)

p = utils.plot_binned(ax[0], time, flux, LC_w.wbins, offs, cmap,
                     ignore_last_bin=False, annotate=False, marker='.',
                     lw=0, mew=0.25, utc=False)

p_res = utils.plot_binned(ax[1], time, resid, LC_w.wbins, offs, 
                          cmap, ignore_last_bin=False, marker='.',
                         lw=0, mew=0.25, annotate=True, utc=False)

for lb, lb_res in zip(p.xaxis.get_ticklabels()[::2], 
                      p_res.xaxis.get_ticklabels()[::2]):
        lb.set_visible(False)
        lb_res.set_visible(False)

IndexError: list index out of range

## Spectra Inspection

### Plot single spectra

In [ ]:
importlib.reload(utils)

fpath = '../WASP43/ut150224/tepspec_final/75nm/LCs_w43_75nm.pkl'
LC = utils.pkl_load(fpath)

fig, ax = plt.subplots(1, 1, figsize=(8,5))
# "zero"
for i in np.arange(0, len(LC['t']), 10): #range(len(LC['t'])):
    utils.plot_spec(ax, LC, i, c='c', show_bins=False) 
#plt.savefig('/Users/mango/Desktop/tepspec_spec_t0.pdf', bboc_inches='tight')

### Wavelength shifts

In [ ]:
importlib.reload(utils)

target = 'WASP43'
date = 'ut170410'
binsize = '75nm'

fpath = f'../{target}/{date}/tepspec_final/{binsize}/LC*.pkl'
fpath = glob.glob(fpath)[0]
#LC = utils.pkl_load(fpath)

# plot
fig, ax = plt.subplots(figsize=(8, 3))

p = utils.plot_spec_diff(ax, LC)

p.set_title(f'{target} {date} {binsize}')
p.legend(ncol=3, loc=3)
p.set_ylim(-4, 4)

### Bin Setup

In [ ]:
for i in np.arange(5300, 9051, 600):
    print(i, i+600)

In [ ]:
# baseline centers + spec vacuum centers (or avg if double peaked)
spec_cen = [5580.0, 5892.9, 6564.6, 7682.0, 8189.0, 8730.0] 

for spec in spec_cen:
    spec_dif = 0
    #print(f'bins for: {spec}')
    for i in range(10):
        spec_dif += 5
        #print(spec-spec_dif, spec+spec_dif, 2*spec_dif)
        print(spec-spec_dif, spec+spec_dif)
    #print('')

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset
importlib.reload(utils)

target = 'WASP43'
date = 'ut170410'
binsize = '75nm'

fpath = f'../{target}/{date}/tepspec_final/{binsize}/LC*.pkl'
fpath = glob.glob(fpath)[0]
#LC = utils.pkl_load(fpath)

fig, ax = plt.subplots()

wbins_path = '../WASP43/ut170410/tepspec_final/bintest/w43_bintest.dat'
utils.plot_spec(ax, LC, 10, wbins_path=wbins_path, c='c')

# inset plots
bin_info = {r'$NaI-D$':{'wav':5892.9, 'loc':2}, 
            r'$H\alpha$':{'wav':6564.6, 'loc':6}, 
            r'$K1_\mathrm{avg}$':{'wav':7682.0, 'loc':1}, 
            r'$NaI-8200_\mathrm{avg}$':{'wav':8189.0, 'loc':7}, 
            'blue baseline':{'wav':5580.0, 'loc':3},
            'red baseline':{'wav':8730.0, 'loc':4}}

# add ylims to each bin:
for bin_name, bin_plot in bin_info.items():
    wavs = LC['spectra']['wavelengths'] # array of interpolated wavelengths
    spectrum = LC['spectra']['WASP43b'][10] # spectrum y-values
    pk_idx, pk_val, bs_val = utils.find_crits(wavs, spectrum, bin_plot['wav'])
    bin_plot['ymin'] = 0.8*pk_val
    bin_plot['ymax'] = bs_val

for bin_name, bin_plot in bin_info.items():
    axins = inset_axes(ax, width=0.75, height=0.75, loc=bin_plot['loc'])
    p  = utils.plot_spec(axins, LC, 10, lw=1, 
                                    wbins_path=wbins_path, c='c')
    #print(bin_name, pk_val, base_val)
    axins.set_xlim(bin_plot['wav'] - 70, bin_plot['wav'] + 70)
    axins.set_title(bin_name, fontsize=8)
    
    axins.set_ylim(bin_plot['ymin'], bin_plot['ymax'])
    axins.axis('off')
    
    if (bin_name == 'blue baseline'):
        mark_inset(ax, axins, loc1=4, loc2=1, fc="none", ec="0.5")
        #axins.set_ylim(36218, 50915)
    if (bin_name == 'red baseline'):
        mark_inset(ax, axins, loc1=3, loc2=2, fc="none", ec="0.5")
        #axins.set_ylim(34951, 49648)

ax.set_xlabel('Wavelength $(\AA)$')
ax.set_ylabel('Integrated Counts')

# ELVIS

## poly and pca WLC

### Corner Plot

#### Plot Function

In [34]:
fpath = '../WASP43/bak/ut170410/poly_WLC/75nm/PD_wl_5300.0_9050.0.pkl'
pd = utils.pkl_load(fpath)
pd.keys()

dict_keys(['q1', 'q2', 'sigma_r', 'sigma_w', 't0', 'a1', 'a0', 'a2', 'RpRs'])

In [149]:
#z = np.empty(shape=(4, 2), type=float)
x = np.array([1, 2, 3, 4])
y = np.array([5, 6, 7, 8])

my_dict = {'a':np.array([x,x]), 'b':np.array([y,y]), 'c':np.array([2*y,2*y])}
my_dict['a'].shape
#sub_dict = {k:v for k, v in my_dict.items() if k in ['a', 'c']}
#np.column_stack((list(my_dict.values())))
#z = np.column_stack((list(sub_dict.values())))
#z = np.column_stack((my_dict['a'], my_dict['b']))
#corner.corner(z)

(2, 4)

In [154]:
len(np.concatenate(list(mcmc['RpRs'].values())))

500000

In [251]:
fpath = '../WASP43/ut170410/poly_WLC/75nm/MCMC_wl_5300.0_9050.0.pkl'
mcmc = utils.pkl_load(fpath)

In [302]:
x = {'b':3, 'a':1}
for k, v in x.items():
    print(k)

b
a


In [333]:
def plot_corner(params, mcmc, **kwargs):  
    # creates another dictionary that just has the entries named in params 
    sub_param_dict = {k:v for k, v in mcmc.items() if k in params}
    # merge all chains into 1 for each param
    for k, v in sub_param_dict.items():
        sub_param_dict[k] = np.concatenate(list(v.values()))
    # order sub_param_dict based on params order and save in param_dict
    param_dict = {p:sub_param_dict[p] for p in params}
    # create (nsamples x nparams) array for corner plot
    samples = np.column_stack((list(param_dict.values())))
    # convert labels to latex math mode
    if 'RpRs' in params: 
        params[params.index('RpRs')] = r'$R_\mathrm{p} / R_\mathrm{s}$'
    if 'a0' in params: 
        params[params.index('a0')] = r'$a_0$'
    if 'a1' in params: 
        params[params.index('a1')] = r'$a_1$'
    if 'a2' in params: 
        params[params.index('a2')] = r'$a_2$'
    if 'sigma_w' in params: 
        params[params.index('sigma_w')] = r'$\sigma_\mathrm{w}$'
    if 'sigma_r' in params: 
        params[params.index('sigma_r')] = r'$\sigma_\mathrm{r}$'
    p = corner.corner(samples, labels=params, show_titles=True, **kwargs)
    return p

In [332]:
#params = ['RpRs', 'a0', 'a1', 'a2', 'q1', 'q2', 'sigma_w', 'sigma_r']
params = ['RpRs', 'sigma_w']
p = plot_corner(params, mcmc, 
                     quantiles=(0.16, 0.84), levels=(1-np.exp(-0.5),))

fpath = '../../'
p.savefig('/Users/mango/Desktop/test.pdf', bbox_inches='tight')

['RpRs', 'sigma_w']
yea


#### Display Plot

In [ ]:
fpath = '../WASP43/ut150309/poly_WLC/75nm/MCMC_wl_5300.0_9800.0.pkl'
mcmc_data = pkl_load(fpath)
poly_params = ['RpRs', 't0', 'a0', 'a1', 'a2', 'sigma_w', 'sigma_r']

plot_corner(poly_params, mcmc_data)
plt.show()

### WLC Plot

In [334]:
fig, ax = plt.subplots(2, 1, figsize=(8, 5), sharex=True, 
                       gridspec_kw = {'height_ratios':[2, 1]})

target = 'WASP43'
date = 'ut150224'
method = 'poly'
binsize = '10nm'
if date == 'ut150224' or date == 'ut150309': filt = 'Spectroscopic2'
else: filt = 'WB5600-9200'

# load data (date: ut, method: poly of pca, binning: wlc)
fpath = f'../WASP43/{date}/{method}_WLC/{binsize}/final_LCs.pkl'
wlc = utils.pkl_load(fpath)

# get stats
WLC = utils.get_detr_stats(wlc)
print(wlc['t0'])
# plot
# the [0] is because there is only one entry in the WLC dictionary
t = WLC.t_idx
ft = WLC.idx_fit
ax[0].plot(t, WLC.model[0], '-', color='r') # model
ax[0].plot(t[ft], WLC.detLC[0][ft], 'ko', alpha=0.25) # detLC
ax[1].plot(t[ft], WLC.resid[0][ft], 'k.', alpha=0.5) # residuals
#ax[1].plot(WLC.t_rel, WLC.resid_elvis, 'bo', alpha=0.5) # ELVIS check

title = '{} -- {} -- {} wavelet WLC -- {} -- Gri-300-17.5 -- {}'
title = title.format(target, date, method, binsize, filt)
if sunspot: title += ' -- sunspot'

ax[0].set_title(title)
fig.tight_layout()
fig.show()

# save figure
dirname = '../journal/Figures/20171220/'
fname = '{}_{}_{}_{}.pdf'.format(date, method, 'WLC', binsize)
#save_fig(dirname, fname) 

2457077.7231232035


In [ ]:
s = 'poly_wavelet_CMC_WLC_sunspot'
#s = 'poly_wavelet_CMC_sunspot_WLC_sunspot'

#s = s.replace('_WLC_sunspot', '')
s = s.replace('_CMC_WLC', '')
s = s.replace('_CMC_sunspot_WLC', '')
s

In [ ]:
print(np.arange(5300, 9000))

#### PCA poly WLC grid plot

In [ ]:
# object info
target = 'WASP43'
date = 'ut150309'
filt = 'Spectroscopic2'
grism = 'Gri-300-17.5'
binning = 'WLC'

methods = ['poly', 'pca']
binsizes = ['10nm', '18nm', '25nm', '75nm']

# plot
fig, ax = plt.subplots(2, 4, figsize=(10,7), sharex=True, sharey=True)

for i, method in enumerate(methods):
    for j, bs in enumerate(binsizes):
        # load data (date: ut, method: poly of pca, binning: wlc)
        fpath = '../WASP43/{}/{}_{}/{}/final_LCs.pkl'
        fpath = fpath.format(date, method, binning, binsize)
        wlc = pkl_load(fpath)
        # get stats
        WLC = get_detr_stats(wlc, use_fitted=True)
        ax[i,j].plot(WLC.t_rel, WLC.detLC[0], 'bo', alpha=0.5) # detrended 
        ax[i,j].plot(WLC.t_rel, WLC.model[0], 'b-') # detrended model
        ax[i,j].plot(WLC.t_rel, WLC.resid[0] + 0.99*np.min(WLC.detLC[0]), 
                     'ro', alpha=0.5) # residuals
        #plot_wlc(ax[i,j], date, method, 'WLC', bs)
        #plot_resid(ax[i,j], date, method, 'WLC', bs, 0.97)

        ax[i,j].set_xlabel(' ')
        ax[i,j].set_ylabel(' ')
        title = '{} wavelet WLC -- {}'.format(method, bs)
        ax[i,j].set_title(title, fontsize=10)

#ax[0][0].set_ylim(0.966, 1.003)

fig.suptitle('WASP43 -- {} -- {} -- {}'.format(date, filt, grism), 
                                               fontsize=12)
fig.text(0.5, 0, '$t-t_0\ (hours)$', ha='center')
fig.text(0, 0.5, 'relative depth', rotation='vertical', va='center')
fig.tight_layout(rect=[0.01, 0.0, 1, 0.95])

# save plot
dirname = '../journal/Figures/20180117/'
fname = '{}_{}_{}_{}.pdf'.format(date, 'poly', 'pca', 'WLC')
#save_fig(dirname, fname)

## Binned plots

In [200]:
fig, ax = plt.subplots(1, 2, figsize=(6,8), sharex=True, sharey=True)
importlib.reload(utils)
date= 'ut150224'
method = 'poly'
#detrending_2 = 'pca'
offs = 0.025
cmap = 'Spectral_r'
binsize = '10nm'
CMC = True
sunspot = False
alpha = 1
mew = 0
hue = 0.7

# load binned pkl
fpath = '../WASP43/bak/{}/{}_binned'.format(date, method)
title = 'WASP43 {} {} wavelet {}:'.format(date, method, binsize)
#fpath = '../WASP43/'+date+'/'+method+'_binned_CMC_sunspot/'+ binsize+'/final_LCs.pkl'
if CMC: 
    fpath += '_CMC'
    title += ' CMC '
if sunspot: 
    fpath += '_sunspot'
    title += ' sunspot '
fpath = '{}/{}/final_LCs.pkl'.format(fpath, binsize)

#title = 'WASP43 -- {}_{} -- {} wavelet binned CMC sunspot'.format(date, 
 #                                                         binsize, method)
    
binned = utils.pkl_load(fpath)
Binned = utils.get_detr_stats(binned)

# poly plots
t_fit = Binned.idx_fit
t_rel = Binned.t_rel[t_fit] 
model = Binned.model
detLC = Binned.detLC[:,t_fit] # filter out bad points, model keeps it (for smoothness)


ax_binned_data = utils.plot_binned(ax[0], t_rel, detLC, Binned.wbins, offs, 
                cmap, marker='.', lw=0, alpha=alpha, mew=mew, annotate=False,
                                 utc=False, hue=hue)

ax_binned_model = utils.plot_binned(ax[0], t_rel, model, 
                                    Binned.wbins, offs,
                              cmap, annotate=False, utc=False)
ax_poly_resid = utils.plot_binned(ax[1], t_rel, 
                                  detLC-model+1, 
                                  Binned.wbins, offs, cmap, marker='.', 
                                  lw=0, alpha=alpha, mew=mew, utc=False,
                                 hue=hue)
ax_poly_resid_base = utils.plot_binned(ax[1], t_rel,
                                       1+np.zeros_like(detLC), 
                                 Binned.wbins, offs, cmap,
                                      utc=False)

ax_binned_model.set_ylabel('normalized flux + const')

fig.tight_layout()

fig.suptitle(title)
fig.text(0.48, 0, '$t-t_0\ (hours)$')
#ax[0].set_ylim(0.968, 1.133)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])

#dirname = '../WASP43/{}/{}_binned/{}/'.format(date, method, binsize)
#dirname = fpath.replace('final_LCs.pkl', '')
#plt.savefig('/Users/mango/Desktop/ut170410.png', bbox_inches='tight', dpi=500)
fname = '{}_{}_{}_{}'.format(date, method, 'binned', binsize)
if CMC: fname += '_CMC'
if sunspot: fname += '_sunspot'
if WLC_sunspot: fname += '_WLC_sunspot'
#fname += '.pdf'
#save: utils.save_fig(dirname, fname)

FileNotFoundError: [Errno 2] No such file or directory: '../WASP43/bak/ut150224/poly_binned_CMC/10nm/final_LCs.pkl'

## Noise fitting

In [ ]:
#with open('/Users/mango/Desktop/ut170410_10nm/')
fpath = '../WASP43/ut150309/pca_binned/final_LCs.pkl'
binned_LC = pkl_load(fpath)

In [ ]:
plt.plot(binned_LC['detLC'][1])

In [ ]:
binned_LC.keys()

In [ ]:
t, t0 = binned_LC['t'], binned_LC['t0']
t_dur = 0.0483 * 24 # from http://exoplanets.org/detail/WASP-43
t_ing = - t_dur/2
t_egr = t_dur/2
time = (t - t0) * 24 # make time relative to t0

# get out of transit (OOT) indices
oot_idx = np.where((time < t_ing) | (time > t_egr))

i = 3
LC_1 = binned_LC['detLC'][i]
plt.plot(time, LC_1, '.', alpha=0.75)
plt.plot(time[oot_idx], LC_1[oot_idx], 'r.')
plt.axvline(t_ing, c='r', ls='--')
plt.axvline(t_egr, c='r', ls='--')

plt.xlabel(r'$t-t_0$ (hours)')
plt.ylabel(r'$F/F_0$')

plt.title('ut170410 $(5300\AA-6050\AA)$')

#plt.savefig('/Users/mango/Desktop/raw.pdf', bbox_inches='tight')

In [ ]:
n = 3
y = LC_1[oot_idx]
x = time[oot_idx]
p, C_p = np.polyfit(x, y, n, cov=True)
t = np.linspace(np.min(x), np.max(x), len(time))
TT = np.vstack([t**(n-i) for i in range(n+1)]).T
yi = np.dot(TT, p)
C_yi = np.dot(TT, np.dot(C_p, TT.T)) # C_y = TT*C_z*TT.T
sig_yi = np.sqrt(np.diag(C_yi))

plt.plot(t, yi, 'r', label='model', alpha=0.25) # combined oot model
plt.plot(x, y, 'ro', label='data', alpha=0.25) # combined oot data

y_corr = (y-yi[oot_idx]) + 1#np.mean(yi)
plt.plot(x, y_corr, 'b.', label='corrected') # oot corrected

it_idx = np.where((time >= t_ing) & (time <= t_egr))
diff_it = yi[it_idx] - 1

#plt.axhline(np.median(y_corr))

plt.xlabel('$(t-t_0)$')
plt.ylabel(r'$F/F_0$')
plt.title('ut170410 $(5300\AA-6050\AA)$')

plt.legend(loc='best')

#plt.savefig('/Users/mango/Desktop/fit.pdf', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(1, 2, sharey=True, figsize=(8,5))
it_idx = np.where((time >= t_ing) & (time <= t_egr))

ax[0].plot(time, LC_1, 'r.', alpha=0.5)
ax[0].set_ylabel(r'$F/F_0$')

ax[1].plot(time[oot_idx], (y-yi[oot_idx]) + 1, 'b.')
ax[1].plot(time[it_idx], LC_1[it_idx]-diff_it, 'b.')
ax[1].plot(time, LC_1, 'r.', alpha=0.25)

ax[1].set_title('filler', color='w') # hacky way to make room for suptitle 
ax[1].set_xlabel('filler', color='w') # hacky way to make room for fig x label

fig.suptitle('ut170410 $(5300\AA-6050\AA)$')
fig.text(0.48, 0, '$t-t_0\ (hours)$')
fig.tight_layout()

#plt.savefig('/Users/mango/Desktop/final.pdf', bbox_inches='tight')

# Transmission Spectra

## $R_p/R_s$ vs. $\lambda$

In [ ]:
importlib.reload(utils)

fig, ax = plt.subplots(figsize=(9,3))

# config plot
date = 'ut170410'
CMC = True
sunspot = True
bin_list = ['75nm', '25nm', '10nm', '18nm']
color_list = ['y', 'b', 'r', 'g'] # color for each binsize plotted
path = '../WASP43/{}/poly_binned'.format(date)
if CMC: path += '_CMC'
if sunspot: path += '_sunspot'
# call the plot
utils.trans_spec_plot(ax, bin_list, color_list, path, 'o')

# set title and ax dimensions 
title = path
    
ax.set_title(title)
ax.set_xlim(5000, 9000)
ax.set_ylim(0.150, 0.164)

# save
fname = 'trans_spec_{}.pdf'.format(path.split('/')[-1])
#utils.save_fig(path, fname)

## ut150224 transmission spectra grid

In [ ]:
def get_path(CMC, sunspot, WLC_sunspot):
    # load binned pkl
    fpath = '../WASP43/{}/{}_binned'.format(date, method)
    title = 'WASP43 {} {}:'.format(date, method)
    #fpath = '../WASP43/'+date+'/'+method+'_binned_CMC_sunspot/'+ binsize+'/final_LCs.pkl'
    if CMC: 
        fpath += '_CMC'
        title += ' CMC '
    if sunspot: 
        fpath += '_sunspot'
        title += ' sunspot '
    if WLC_sunspot and CMC: 
        fpath += '_WLC_sunspot'
        title += ' WLC_sunspot '
    #fpath = '{}/{}'.format(fpath, binsize)
    
    return fpath, title 

In [ ]:
fig, axes = plt.subplots(3, 2, sharex=True, sharey=True, figsize=(10,6))
axes = axes.flatten()
date = 'ut150224'
paths = [[False, False, False], [False, True, False],
         [True, False, False],  [True, True, False],
         [True, False, True],   [True, True, True]]

bin_list = ['75nm']
color_list = ['y']
#for path in paths:
for ax, path in zip(axes, paths):
    fpath, title = get_path(CMC=path[0], sunspot=path[1], WLC_sunspot=path[2])
    #print(fpath)
    a = trans_spec_plot(ax, bin_list, color_list, fpath, 'o')
    a.set_title(title)
    
fig.tight_layout()
dirname = '../../mercedes-group/meeting/20181603'
fname = 'ut150224_spec_grid.pdf'
utils.save_fig(dirname, fname)

## $R_p / R_s$ vs. binsize 

In [227]:
importlib.reload(utils)
date= 'ut170410'
method = 'poly'
binsize = 'bintest'
CMC = True
sunspot = False

fpath = f'../WASP43/{date}/{method}_binned'

title = f'WASP43 {date} {method} wavelet {binsize}:'
if CMC: 
    fpath += '_CMC'
    title += ' CMC '
if sunspot: 
    fpath += '_sunspot'
    title += ' sunspot '

fpath = f'{fpath}/{binsize}/posteriors.pkl'
# load up stats data
post = utils.pkl_load(fpath)
post.keys()

# initialize dictionary that holds summary stats of each bin
# e.g. median, LCI, and UCI of RpRs
# keeping these separate for easy plotting later
names_species = ['$NaI-D$', r'$H\alpha$', '$K1\_$avg', '$NaI-8200\_avg$']
centers_species = [5892.9, 6564.6, 7682.0, 8189.0]
widths_species = [42, 12, 45, 34]
names_baseline = ['blue baseline', 'red baseline']
centers_baseline = [5580.0, 8730.0]
widths_baseline = [0, 0]

names = names_species + names_baseline
centers = centers_species + centers_baseline
widths = widths_species + widths_baseline
bin_info = {}
for name, center, width in zip(names, centers, widths):
    bin_info[name] = {'center':center, 'width':width, 'RpRs':{}}

# fill in the dictionary
for wl_min, wl_max, rprs, rprs_p, rprs_m in zip(
    post['wl_mins'], post['wl_maxs'], 
    post['RpRs']['meds'], post['RpRs']['UCIs'], post['RpRs']['LCIs']):
    l, r = wl_min, wl_max
    bin_cen = (r + l) / 2
    bin_size = r - l
    for k, v in bin_info.items():
        if bin_info[k]['center'] == bin_cen: 
            bin_info[k]['RpRs'][bin_size] = {'med':rprs, 
                                             'UCI':rprs_p,
                                             'LCI':rprs_m}

# split (well, copy) into separate species and baseline dict
bin_info_species = {k:bin_info[k] for k in names_species}
bin_info_baseline = {k:bin_info[k] for k in names_baseline}

# and plot
fig, axes = plt.subplots(2, 2, sharex=True, sharey=True)

for ax, name_species in zip(axes.flatten(), bin_info_species.keys()):
    utils.plot_bintest(ax, bin_info_species, name_species, c='g')
    utils.plot_bintest(ax, bin_info_baseline, 'blue baseline', c='b')
    utils.plot_bintest(ax, bin_info_baseline, 'red baseline', c='r')
    # over plot species widths
    ax.axvline(bin_info_species[name_species]['width'], ls='--', 
               c='k', alpha=0.25)
    ax.set_title(name_species)
    
fig.suptitle(title)
fig.tight_layout(rect=[0.01, 0.0, 1, 0.95])

dirname = '../meeting/20180404'
fname = f'{title}.pdf'
utils.save_fig(dirname, fname)